In [27]:
import sys, csv
sys.path.append('../registrar_reports/src')
from parts_parser import parse

import pandas as pd

In [69]:
raw_file  = pd.read_csv('../output/impression_count/raw.csv', index_col=0)

In [61]:
def clean_data(df):
    # rename modality of DX and PX to CR
    df["modality"] = df["modality"].apply(lambda x: "CR" if x in ["DX", "PX"] else x)
    return df

In [60]:
def deduplication(df):
    # Split rows into formal reports and impressions
    formal_reports = raw_file[raw_file['action'] == 'Complete dictation']
    impressions = raw_file[raw_file['action'] == 'Add Emergency Impression']

    # Only keep impressions that have unique accession_no
    impressions = impressions.drop_duplicates(subset="accession_no")

    # Remove impressions that are already in formal reports
    impressions = impressions[
        ~impressions["accession_no"].isin(formal_reports["accession_no"])
    ]

    # Combine the two dataframes
    deduped = pd.concat([formal_reports, impressions])
    return deduped

In [63]:
def parse_xrays(df):
    # Parse the parts of the report
    df["sum_parts"] = df.apply(
        lambda row: parse(row.descriptor) if row.modality == "CR" else 1, axis=1
    )
    return df

In [70]:
deduped = deduplication(clean_data(raw_file))

In [85]:
def export_data(df):
    results = []
    for user_and_action, user_group in deduped.groupby(["user", "action"]):
        user_result = {"username": user_and_action[0], "action": user_and_action[1]}
        parsed = parse_xrays(user_group)
        for modality, modality_group in parsed.groupby("modality"):
            user_result[str(modality)] = modality_group["sum_parts"].sum()
        results.append(user_result)
        # append username, modality, sum_parts to result
    # make NaN 0 (integer)
    results = pd.DataFrame(results).fillna(0)
    return results

    

In [86]:
print(export_data(deduped))

            username                    action   CR   CT  MR   US    XA
0  Shi, Tubo (TuboS)  Add Emergency Impression   10  194   9    1   0.0
1  Shi, Tubo (TuboS)        Complete dictation  835  289  73  216  15.0
